# Deferred After the inputs have been created

The input_df_dict dictionary contains the following information (keys)
 - billings
 - ADBE_cal
 - bookings
 - FX_forwards
 - FX_rates


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression

In [2]:
input_df_dict = pickle.load(open('../data/processed/all_inputs.p', 'rb'))

df_billings = input_df_dict['billings']
df_ADBE_cal = input_df_dict['ADBE_cal']
df_bookings = input_df_dict['bookings']
df_FX_fwds  = input_df_dict['FX_forwards']
df_FX_rates = input_df_dict['FX_rates']

In [6]:
df_bookings.tail(20)


,BU,segment,product,geo,country,booking_type,Quarter,US_amount,Country,Currency
5994,Experience Cloud,Adobe Experience Manager,Forms,AMER,United States,Total Subscription Attrition,Q3 2020,4.923126e+01,United States,USD
5995,Experience Cloud,Adobe Experience Manager,Forms,AMER,United States,Total Subscription Attrition,Q3 2020,-4.923126e+01,United States,USD
5996,Experience Cloud,Adobe Experience Manager,Forms,AMER,United States,Total Subscription Attrition,Q4 2020,4.991613e+01,United States,USD
5997,Experience Cloud,Adobe Experience Manager,Forms,AMER,United States,Total Subscription Attrition,Q4 2020,-4.991613e+01,United States,USD
5998,Experience Cloud,Magento,Magento,EMEA,Germany,Consulting,Q1 2020,1.542400e+06,Germany,EUR
5999,Experience Cloud,Magento,Magento,EMEA,United Kingdom,Consulting,Q1 2020,3.856000e+05,United Kingdom,GBP
6000,Experience Cloud,Magento,Magento,EMEA,Germany,Consulting,Q2 2020,1.950400e+06,Germany,EUR
6001,Experience Cloud,Magento,Magento,EMEA,United Kingdom,Consulting,Q2 2020,4.876000e+05,United Kingdom,GBP
6002,Experience Cloud,Magento,Magento,EMEA,Germany,Consulting,Q3 2020,1.926400e+06,Germany,EUR
6003,Experience Cloud,Magento,Magento,EMEA,United Kingdom,Consulting,Q3 2020,4.816000e+05,United Kingdom,GBP


## We need to split the bookings information from quarters into periods
For each currency and BU, we will take the percentage of annual bookings for each period in a quarter.
This percentage will be what we use to split the bookings into periods


In [7]:
# find the last period in the billings index
last_period = '2020-03'


In [8]:
list_BUs = df_bookings['BU'].unique()
list_curr = df_bookings['Currency'].unique()

In [9]:
list_BUs

array(['Experience Cloud', 'Creative', 'Document Cloud'], dtype=object)

In [10]:
# creating dataframe of zeros

l_BU = []
l_curr = []
for BU in list_BUs:
    for curr in list_curr:
        l_BU.append(BU)
        l_curr.append(curr)
print(l_BU)
print(l_curr)
l_zero = np.arange(len(l_BU))

['Experience Cloud', 'Experience Cloud', 'Experience Cloud', 'Experience Cloud', 'Experience Cloud', 'Creative', 'Creative', 'Creative', 'Creative', 'Creative', 'Document Cloud', 'Document Cloud', 'Document Cloud', 'Document Cloud', 'Document Cloud']
['AUD', 'EUR', 'USD', 'JPY', 'GBP', 'AUD', 'EUR', 'USD', 'JPY', 'GBP', 'AUD', 'EUR', 'USD', 'JPY', 'GBP']


In [11]:
list_curr

array(['AUD', 'EUR', 'USD', 'JPY', 'GBP'], dtype=object)

In [12]:
l_zero = np.zeros(len(l_BU))
l_zero

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
data= {'BU':l_BU, 'curr':l_curr, 
      'Q1':l_zero,
      'Q2':l_zero,
      'Q3':l_zero,
      'Q4':l_zero,
      'P01':l_zero,
      'P02':l_zero,
      'P03':l_zero,
      'P04':l_zero,
      'P05':l_zero,
      'P06':l_zero,
      'P07':l_zero,
       'P08':l_zero,
       'P09':l_zero,
       'P10':l_zero,
       'P11':l_zero,
       'P12':l_zero,
      }

df_book_period=pd.DataFrame(data)

In [14]:
df_book_period

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11,P12
0,Experience Cloud,AUD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Experience Cloud,EUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Experience Cloud,USD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Experience Cloud,JPY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Experience Cloud,GBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Creative,AUD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Creative,EUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Creative,USD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Creative,JPY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Creative,GBP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_bookings.head(10)

,BU,segment,product,geo,country,booking_type,Quarter,US_amount,Country,Currency
0,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q1 2020,-243336.527786,Aus and New Zealand,AUD
1,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q1 2020,-309237.732096,Aus and New Zealand,AUD
2,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q1 2020,-61767.378056,Aus and New Zealand,AUD
3,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q1 2020,-503969.121094,Aus and New Zealand,AUD
4,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q2 2020,-145084.185190,Aus and New Zealand,AUD
5,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q2 2020,-184376.364697,Aus and New Zealand,AUD
6,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q2 2020,-36827.474272,Aus and New Zealand,AUD
7,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q2 2020,-325745.459992,Aus and New Zealand,AUD
8,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q3 2020,-103805.125604,Aus and New Zealand,AUD
9,Experience Cloud,Marketo,Marketo,ASIA,Aus and New Zealand,Total Subscription Attrition,Q3 2020,-131917.973491,Aus and New Zealand,AUD


In [16]:
# fill in the quarters
for i in range(len(df_book_period['BU'])):
    
    this_BU = df_book_period['BU'][i]
    this_curr = df_book_period['curr'][i]
    this_slice = df_bookings[(df_bookings['BU']==this_BU)&
                          (df_bookings['Currency']==this_curr)]
    
    this_Q1= this_slice[this_slice['Quarter']=='Q1 2020']
    sum_Q1 = this_Q1['US_amount'].sum()
    df_book_period['Q1'].loc[i]=sum_Q1
    
    this_Q2= this_slice[this_slice['Quarter']=='Q2 2020']
    sum_Q2 = this_Q2['US_amount'].sum()
    df_book_period['Q2'].loc[i]=sum_Q2
    
    this_Q3= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q3 = this_Q3['US_amount'].sum()
    df_book_period['Q3'].loc[i]=sum_Q3
    
    this_Q4= this_slice[this_slice['Quarter']=='Q4 2020']
    sum_Q4 = this_Q4['US_amount'].sum()
    df_book_period['Q4'].loc[i]=sum_Q4
    
    

/Users/davidsmith/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
df_book_period

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11,P12
0,Experience Cloud,AUD,7.433434e+06,1.358439e+07,1.816287e+07,1.816287e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Experience Cloud,EUR,3.499845e+07,3.721689e+07,7.695415e+07,7.695415e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Experience Cloud,USD,1.247171e+08,1.986764e+08,3.135637e+08,3.135637e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Experience Cloud,JPY,1.182163e+07,2.024655e+07,2.681428e+07,2.681428e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Experience Cloud,GBP,2.157488e+07,2.645604e+07,4.759333e+07,4.759333e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Creative,AUD,1.393692e+07,1.379181e+07,2.364523e+07,2.364523e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Creative,EUR,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Creative,USD,2.075290e+07,1.613099e+07,3.783942e+07,3.783942e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Creative,JPY,2.711939e+06,6.155972e+06,5.318174e+06,5.318174e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Creative,GBP,1.579663e+07,1.104858e+07,2.342099e+07,2.342099e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# list of quarters for the percentages
list_q2 = ['2019-04', '2019-05', '2019-06']
list_q3 = ['2019-07', '2019-08', '2019-09']
list_q4 = ['2019-10', '2019-11', '2019-12']
list_q1 = [ '2020-01', '2020-02', '2020-03']

list_periods = ['2020-01', '2020-02', '2020-03',
                '2019-04', '2019-05', '2019-06',
                '2019-07', '2019-08', '2019-09',
                '2019-10', '2019-11', '2019-12']
print(list_periods)
list_p_headers = ['P01', 'P02', 'P03',
                  'P04', 'P05', 'P06',
                  'P07', 'P08', 'P09',
                  'P10', 'P11', 'P12'
                 ]
list_q_headers = ['Q1', 'Q1', 'Q1',
                  'Q2', 'Q2', 'Q2',
                  'Q3', 'Q3', 'Q3',
                  'Q4', 'Q4', 'Q4']
print(list_p_headers)

['2020-01', '2020-02', '2020-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12']
['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12']


In [19]:
# fill in the periods
for i in range(len(df_book_period['BU'])):
    
    this_BU = df_book_period['BU'][i]
    this_curr = df_book_period['curr'][i]
    
    this_slice = df_billings[(df_billings['BU']==this_BU)&
                          (df_billings['curr']==this_curr)]
    
    for j in range(len(list_periods)):
        this_period = list_periods[j]
        this_header = list_p_headers[j]
        this_quarter = list_q_headers[j]
        this_P_slice = this_slice[this_slice['period']==this_period]
        df_book_period.loc[[i],[this_header]]=this_P_slice['deferred_1Y_DC'].sum()
        
df_book_period['bill_Q1_sum'] = df_book_period['P01'] + df_book_period['P02'] + df_book_period['P03']    
df_book_period['bill_Q2_sum'] = df_book_period['P04'] + df_book_period['P05'] + df_book_period['P06']    
df_book_period['bill_Q3_sum'] = df_book_period['P07'] + df_book_period['P08'] + df_book_period['P09']    
df_book_period['bill_Q4_sum'] = df_book_period['P10'] + df_book_period['P11'] + df_book_period['P12']    

df_book_period['P01'] = df_book_period['Q1']*df_book_period['P01']/df_book_period['bill_Q1_sum']
df_book_period['P02'] = df_book_period['Q1']*df_book_period['P02']/df_book_period['bill_Q1_sum']
df_book_period['P03'] = df_book_period['Q1']*df_book_period['P03']/df_book_period['bill_Q1_sum']

df_book_period['P04'] = df_book_period['Q2']*df_book_period['P04']/df_book_period['bill_Q2_sum']
df_book_period['P05'] = df_book_period['Q2']*df_book_period['P05']/df_book_period['bill_Q2_sum']
df_book_period['P06'] = df_book_period['Q2']*df_book_period['P06']/df_book_period['bill_Q2_sum']

df_book_period['P07'] = df_book_period['Q3']*df_book_period['P07']/df_book_period['bill_Q3_sum']
df_book_period['P08'] = df_book_period['Q3']*df_book_period['P08']/df_book_period['bill_Q3_sum']
df_book_period['P09'] = df_book_period['Q3']*df_book_period['P09']/df_book_period['bill_Q3_sum']

df_book_period['P10'] = df_book_period['Q4']*df_book_period['P10']/df_book_period['bill_Q4_sum']
df_book_period['P11'] = df_book_period['Q4']*df_book_period['P11']/df_book_period['bill_Q4_sum']
df_book_period['P12'] = df_book_period['Q4']*df_book_period['P12']/df_book_period['bill_Q4_sum']

        

In [20]:
df_book_period.loc[0]['Q1']

7433433.702077989

In [21]:
df_book_period

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,...,P07,P08,P09,P10,P11,P12,bill_Q1_sum,bill_Q2_sum,bill_Q3_sum,bill_Q4_sum
0,Experience Cloud,AUD,7.433434e+06,1.358439e+07,1.816287e+07,1.816287e+07,2.446926e+06,2.411696e+06,2.574812e+06,1.902455e+06,...,2.811683e+06,7.213659e+06,8.137530e+06,4.766754e+06,4.182153e+06,9.213966e+06,2.333180e+07,9.811537e+06,2.094163e+07,2.504277e+07
1,Experience Cloud,EUR,3.499845e+07,3.721689e+07,7.695415e+07,7.695415e+07,7.586985e+06,1.835026e+07,9.061199e+06,9.100126e+06,...,3.280011e+07,2.194373e+07,2.221031e+07,2.872638e+07,1.689213e+07,3.133564e+07,6.389787e+07,3.129786e+07,3.227531e+07,4.630633e+07
2,Experience Cloud,USD,1.247171e+08,1.986764e+08,3.135637e+08,3.135637e+08,4.488347e+07,4.657423e+07,3.325938e+07,4.861151e+07,...,8.742538e+07,7.658396e+07,1.495543e+08,1.049945e+08,9.468008e+07,1.138891e+08,3.555219e+08,1.412866e+08,2.118878e+08,2.349536e+08
3,Experience Cloud,JPY,1.182163e+07,2.024655e+07,2.681428e+07,2.681428e+07,4.191875e+06,4.971201e+06,2.658557e+06,7.899640e+05,...,1.008977e+07,1.120458e+07,5.519922e+06,5.585768e+06,1.139248e+07,9.836033e+06,1.446551e+09,2.967100e+09,1.280155e+09,1.099353e+09
4,Experience Cloud,GBP,2.157488e+07,2.645604e+07,4.759333e+07,4.759333e+07,5.511899e+06,6.248136e+06,9.814842e+06,1.102075e+07,...,2.166839e+07,1.497688e+07,1.094806e+07,1.556676e+07,1.121560e+07,2.081097e+07,1.886141e+07,1.375987e+07,1.558331e+07,2.506557e+07
5,Creative,AUD,1.393692e+07,1.379181e+07,2.364523e+07,2.364523e+07,4.502145e+06,3.363263e+06,6.071509e+06,3.892268e+06,...,6.937723e+06,7.857480e+06,8.850029e+06,5.861274e+06,5.146632e+06,1.263733e+07,3.256804e+07,3.351668e+07,3.974319e+07,4.712909e+07
6,Creative,EUR,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.632360e+08,1.348083e+08,1.558901e+08,2.169764e+08
7,Creative,USD,2.075290e+07,1.613099e+07,3.783942e+07,3.783942e+07,7.004166e+06,5.285728e+06,8.463003e+06,4.495379e+06,...,1.041130e+07,9.500524e+06,1.792759e+07,1.064223e+07,1.080927e+07,1.638791e+07,4.939587e+08,4.483563e+08,5.794996e+08,5.868422e+08
8,Creative,JPY,2.711939e+06,6.155972e+06,5.318174e+06,5.318174e+06,7.529930e+05,6.734288e+05,1.285517e+06,1.648482e+06,...,1.427165e+06,1.766215e+06,2.124794e+06,1.646169e+06,1.148706e+06,2.523300e+06,9.671118e+09,1.286305e+10,1.101781e+10,1.018881e+10
9,Creative,GBP,1.579663e+07,1.104858e+07,2.342099e+07,2.342099e+07,5.636788e+06,4.382726e+06,5.777112e+06,3.215084e+06,...,5.618554e+06,5.051653e+06,1.275078e+07,7.468475e+06,5.897097e+06,1.005542e+07,3.179986e+07,2.861961e+07,4.507428e+07,3.930805e+07


In [22]:
df_book_period.columns

Index(['BU', 'curr', 'Q1', 'Q2', 'Q3', 'Q4', 'P01', 'P02', 'P03', 'P04', 'P05',
       'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'bill_Q1_sum',
       'bill_Q2_sum', 'bill_Q3_sum', 'bill_Q4_sum'],
      dtype='object')

In [23]:
df_book_period.drop(['bill_Q1_sum', 'bill_Q2_sum', 'bill_Q3_sum', 'bill_Q4_sum'], axis=1,inplace=True)

## Now working on converting these to local currency

In [24]:
df_FX_fwds.set_index('curr', inplace=True)

In [25]:
EUR = df_FX_fwds.loc['EUR','forward']
EUR

0.917349

In [26]:
list_fwds =[]
for i in range(len(df_book_period['curr'])):
    this_curr = df_book_period['curr'][i]
    print(this_curr)
    if this_curr == 'USD':
        this_fwd=1
    else:
        this_fwd = df_FX_fwds.loc[this_curr, 'forward']
    
    print(this_fwd)
    list_fwds.append(this_fwd)
df_book_period['FX_fwd_rate'] = list_fwds

AUD
1.49115
EUR
0.917349
USD
1
JPY
108.9682
GBP
0.766329
AUD
1.49115
EUR
0.917349
USD
1
JPY
108.9682
GBP
0.766329
AUD
1.49115
EUR
0.917349
USD
1
JPY
108.9682
GBP
0.766329


In [27]:
df_book_period

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11,P12,FX_fwd_rate
0,Experience Cloud,AUD,7.433434e+06,1.358439e+07,1.816287e+07,1.816287e+07,2.446926e+06,2.411696e+06,2.574812e+06,1.902455e+06,4.116428e+06,7.565509e+06,2.811683e+06,7.213659e+06,8.137530e+06,4.766754e+06,4.182153e+06,9.213966e+06,1.491150
1,Experience Cloud,EUR,3.499845e+07,3.721689e+07,7.695415e+07,7.695415e+07,7.586985e+06,1.835026e+07,9.061199e+06,9.100126e+06,1.816391e+07,9.952856e+06,3.280011e+07,2.194373e+07,2.221031e+07,2.872638e+07,1.689213e+07,3.133564e+07,0.917349
2,Experience Cloud,USD,1.247171e+08,1.986764e+08,3.135637e+08,3.135637e+08,4.488347e+07,4.657423e+07,3.325938e+07,4.861151e+07,7.441966e+07,7.564527e+07,8.742538e+07,7.658396e+07,1.495543e+08,1.049945e+08,9.468008e+07,1.138891e+08,1.000000
3,Experience Cloud,JPY,1.182163e+07,2.024655e+07,2.681428e+07,2.681428e+07,4.191875e+06,4.971201e+06,2.658557e+06,7.899640e+05,1.747340e+07,1.983193e+06,1.008977e+07,1.120458e+07,5.519922e+06,5.585768e+06,1.139248e+07,9.836033e+06,108.968200
4,Experience Cloud,GBP,2.157488e+07,2.645604e+07,4.759333e+07,4.759333e+07,5.511899e+06,6.248136e+06,9.814842e+06,1.102075e+07,8.763335e+06,6.671956e+06,2.166839e+07,1.497688e+07,1.094806e+07,1.556676e+07,1.121560e+07,2.081097e+07,0.766329
5,Creative,AUD,1.393692e+07,1.379181e+07,2.364523e+07,2.364523e+07,4.502145e+06,3.363263e+06,6.071509e+06,3.892268e+06,3.451427e+06,6.448112e+06,6.937723e+06,7.857480e+06,8.850029e+06,5.861274e+06,5.146632e+06,1.263733e+07,1.491150
6,Creative,EUR,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.917349
7,Creative,USD,2.075290e+07,1.613099e+07,3.783942e+07,3.783942e+07,7.004166e+06,5.285728e+06,8.463003e+06,4.495379e+06,4.273529e+06,7.362081e+06,1.041130e+07,9.500524e+06,1.792759e+07,1.064223e+07,1.080927e+07,1.638791e+07,1.000000
8,Creative,JPY,2.711939e+06,6.155972e+06,5.318174e+06,5.318174e+06,7.529930e+05,6.734288e+05,1.285517e+06,1.648482e+06,2.206511e+06,2.300979e+06,1.427165e+06,1.766215e+06,2.124794e+06,1.646169e+06,1.148706e+06,2.523300e+06,108.968200
9,Creative,GBP,1.579663e+07,1.104858e+07,2.342099e+07,2.342099e+07,5.636788e+06,4.382726e+06,5.777112e+06,3.215084e+06,3.079131e+06,4.754361e+06,5.618554e+06,5.051653e+06,1.275078e+07,7.468475e+06,5.897097e+06,1.005542e+07,0.766329


In [28]:
df_book_period['P01_DC']=df_book_period['P01']* df_book_period['FX_fwd_rate']
df_book_period['P02_DC']=df_book_period['P02']* df_book_period['FX_fwd_rate']
df_book_period['P03_DC']=df_book_period['P03']* df_book_period['FX_fwd_rate']
df_book_period['P04_DC']=df_book_period['P04']* df_book_period['FX_fwd_rate']
df_book_period['P05_DC']=df_book_period['P05']* df_book_period['FX_fwd_rate']
df_book_period['P06_DC']=df_book_period['P06']* df_book_period['FX_fwd_rate']
df_book_period['P07_DC']=df_book_period['P07']* df_book_period['FX_fwd_rate']
df_book_period['P08_DC']=df_book_period['P08']* df_book_period['FX_fwd_rate']
df_book_period['P09_DC']=df_book_period['P09']* df_book_period['FX_fwd_rate']
df_book_period['P10_DC']=df_book_period['P10']* df_book_period['FX_fwd_rate']
df_book_period['P11_DC']=df_book_period['P11']* df_book_period['FX_fwd_rate']
df_book_period['P12_DC']=df_book_period['P12']* df_book_period['FX_fwd_rate']

In [29]:
df_book_period

,BU,curr,Q1,Q2,Q3,Q4,P01,P02,P03,P04,...,P03_DC,P04_DC,P05_DC,P06_DC,P07_DC,P08_DC,P09_DC,P10_DC,P11_DC,P12_DC
0,Experience Cloud,AUD,7.433434e+06,1.358439e+07,1.816287e+07,1.816287e+07,2.446926e+06,2.411696e+06,2.574812e+06,1.902455e+06,...,3.839431e+06,2.836845e+06,6.138212e+06,1.128131e+07,4.192641e+06,1.075665e+07,1.213428e+07,7.107945e+06,6.236217e+06,1.373940e+07
1,Experience Cloud,EUR,3.499845e+07,3.721689e+07,7.695415e+07,7.695415e+07,7.586985e+06,1.835026e+07,9.061199e+06,9.100126e+06,...,8.312282e+06,8.347991e+06,1.666265e+07,9.130243e+06,3.008915e+07,2.013006e+07,2.037461e+07,2.635211e+07,1.549598e+07,2.874572e+07
2,Experience Cloud,USD,1.247171e+08,1.986764e+08,3.135637e+08,3.135637e+08,4.488347e+07,4.657423e+07,3.325938e+07,4.861151e+07,...,3.325938e+07,4.861151e+07,7.441966e+07,7.564527e+07,8.742538e+07,7.658396e+07,1.495543e+08,1.049945e+08,9.468008e+07,1.138891e+08
3,Experience Cloud,JPY,1.182163e+07,2.024655e+07,2.681428e+07,2.681428e+07,4.191875e+06,4.971201e+06,2.658557e+06,7.899640e+05,...,2.896981e+08,8.608096e+07,1.904045e+09,2.161050e+08,1.099464e+09,1.220943e+09,6.014960e+08,6.086711e+08,1.241418e+09,1.071815e+09
4,Experience Cloud,GBP,2.157488e+07,2.645604e+07,4.759333e+07,4.759333e+07,5.511899e+06,6.248136e+06,9.814842e+06,1.102075e+07,...,7.521398e+06,8.445520e+06,6.715598e+06,5.112913e+06,1.660511e+07,1.147722e+07,8.389813e+06,1.192926e+07,8.594836e+06,1.594805e+07
5,Creative,AUD,1.393692e+07,1.379181e+07,2.364523e+07,2.364523e+07,4.502145e+06,3.363263e+06,6.071509e+06,3.892268e+06,...,9.053531e+06,5.803955e+06,5.146595e+06,9.615103e+06,1.034519e+07,1.171668e+07,1.319672e+07,8.740039e+06,7.674401e+06,1.884415e+07
6,Creative,EUR,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,Creative,USD,2.075290e+07,1.613099e+07,3.783942e+07,3.783942e+07,7.004166e+06,5.285728e+06,8.463003e+06,4.495379e+06,...,8.463003e+06,4.495379e+06,4.273529e+06,7.362081e+06,1.041130e+07,9.500524e+06,1.792759e+07,1.064223e+07,1.080927e+07,1.638791e+07
8,Creative,JPY,2.711939e+06,6.155972e+06,5.318174e+06,5.318174e+06,7.529930e+05,6.734288e+05,1.285517e+06,1.648482e+06,...,1.400805e+08,1.796321e+08,2.404396e+08,2.507335e+08,1.555156e+08,1.924612e+08,2.315350e+08,1.793800e+08,1.251724e+08,2.749595e+08
9,Creative,GBP,1.579663e+07,1.104858e+07,2.342099e+07,2.342099e+07,5.636788e+06,4.382726e+06,5.777112e+06,3.215084e+06,...,4.427168e+06,2.463812e+06,2.359627e+06,3.643405e+06,4.305661e+06,3.871228e+06,9.771293e+06,5.723309e+06,4.519116e+06,7.705757e+06


In [30]:
df_book_period.columns

Index(['BU', 'curr', 'Q1', 'Q2', 'Q3', 'Q4', 'P01', 'P02', 'P03', 'P04', 'P05',
       'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'FX_fwd_rate',
       'P01_DC', 'P02_DC', 'P03_DC', 'P04_DC', 'P05_DC', 'P06_DC', 'P07_DC',
       'P08_DC', 'P09_DC', 'P10_DC', 'P11_DC', 'P12_DC'],
      dtype='object')

# Ready to build the deferred forecast for existing billings to continue

In [31]:
df_billings.sample(10)


,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,deferred_1M_DC,...,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,Period_Weeks
4070,AUD,DX Other,2016-07,0.00,0.00,0.00,0.00,0.0,0.00,1.868953e+04,...,0.000000e+00,0.0,0.0,1.868953e+04,0.000000,0.000000,0.000000e+00,0.00,0.0,4
277,AUD,Print & Publishing,2015-12,145259.00,103858.30,0.00,0.00,0.0,0.00,2.695538e+04,...,6.510307e+05,27018.0,0.0,1.926093e+04,0.000000,0.000000,4.643727e+05,19317.94,0.0,5
1504,GBP,Document Cloud,2017-10,519657.75,689985.15,0.00,0.00,0.0,0.00,1.872939e+05,...,1.242754e+06,23492.1,0.0,2.476384e+05,9763.536667,-15.794044,1.650995e+06,31666.24,0.0,4
368,CAD,Experience Cloud,2015-05,561210.72,454002.58,18172.00,14558.19,58700.0,47053.37,0.000000e+00,...,7.790039e+05,0.0,0.0,0.000000e+00,0.000000,0.000000,6.227299e+05,0.00,0.0,4
2656,SEK,Experience Cloud,2017-03,-102006.37,-11456.72,3425495.61,381875.83,-2.1,-0.23,0.000000e+00,...,6.418621e+06,0.0,0.0,0.000000e+00,70252.390000,221.900000,7.173804e+05,0.00,0.0,5
679,CHF,Print & Publishing,2015-01,3939.00,4056.18,0.00,0.00,0.0,0.00,1.463000e+03,...,0.000000e+00,0.0,0.0,1.509890e+03,0.000000,0.000000,0.000000e+00,0.00,0.0,4
4066,TRY,Document Cloud,2020-02,0.00,0.00,0.00,0.00,0.0,0.00,0.000000e+00,...,3.810000e+02,0.0,0.0,0.000000e+00,0.000000,0.000000,6.451000e+01,0.00,0.0,4
1801,INR,Creative,2020-02,138900.71,1946.97,0.00,0.00,0.0,0.00,3.890546e+07,...,1.400479e+07,0.0,0.0,5.456988e+05,0.000000,0.000000,1.964611e+05,0.00,0.0,4
496,CHF,Creative,2019-05,39070.00,39037.99,0.00,0.00,0.0,0.00,2.645482e+06,...,9.728125e+05,0.0,0.0,2.642291e+06,0.000000,0.000000,9.715696e+05,0.00,0.0,4
585,CHF,Experience Cloud,2016-07,36504.38,37832.41,118438.75,123351.59,53794.0,55557.82,0.000000e+00,...,6.935714e+05,0.0,0.0,0.000000e+00,0.000000,15622.200000,7.175252e+05,0.00,0.0,4


## Forecasting the billings into the future
### Steps
 - create list of bill periods that is sorted for the lookup functions
 - create forecast dataframe that includes the same columns (though in document currency) for the billings
 - add the bookings forecast to this data
 - create impact on deferred (project the new waterfall from this_
 - load up accounting's version of the initial waterfall (by BU)
 - reporting

In [32]:
list_bill_periods = df_billings['period'].unique()
list_bill_periods.sort()
print(list_bill_periods)

['2015-01' '2015-02' '2015-03' '2015-04' '2015-05' '2015-06' '2015-07'
 '2015-08' '2015-09' '2015-10' '2015-11' '2015-12' '2016-01' '2016-02'
 '2016-03' '2016-04' '2016-05' '2016-06' '2016-07' '2016-08' '2016-09'
 '2016-10' '2016-11' '2016-12' '2017-01' '2017-02' '2017-03' '2017-04'
 '2017-05' '2017-06' '2017-07' '2017-08' '2017-09' '2017-10' '2017-11'
 '2017-12' '2018-01' '2018-02' '2018-03' '2018-04' '2018-05' '2018-06'
 '2018-07' '2018-08' '2018-09' '2018-10' '2018-11' '2018-12' '2019-01'
 '2019-02' '2019-03' '2019-04' '2019-05' '2019-06' '2019-07' '2019-08'
 '2019-09' '2019-10' '2019-11' '2019-12' '2020-01' '2020-02' '2020-03']


In [33]:
df_billings['curr'].unique()

array(['ARS', 'AUD', 'BRL', 'CAD', 'CHF', 'CLP', 'COP', 'DKK', 'EUR',
       'GBP', 'HKD', 'INR', 'JPY', 'KRW', 'NOK', 'NZD', 'PEN', 'PHP',
       'RUB', 'SEK', 'SGD', 'THB', 'TWD', 'USD', 'IDR', 'ILS', 'MYR',
       'TRY', 'MXP'], dtype=object)

In [34]:
df_billings['BU'].unique()

array(['Creative', 'Document Cloud', 'Experience Cloud', 'LiveCycle',
       'Other Solutions', 'Print & Publishing', 'DX Other'], dtype=object)

In [35]:
v_BU = df_billings['BU'].copy()
v_curr = df_billings['curr'].copy()
v_both = v_BU + v_curr
v_unique = v_both.unique()

In [36]:
type(v_unique)

numpy.ndarray

In [37]:
v_un_BU = [sub[:-3] for sub in v_unique]
v_un_curr = [sub[-3:] for sub in v_unique]

In [38]:
list_future_periods = ['2020-04', '2020-05', '2020-06',
                       '2020-07', '2020-08', '2020-09',
                       '2020-10', '2020-11', '2020-12',
                       '2021-01', '2021-02', '2021-03']

In [39]:
# creating the vectors for the future billings dataframe
v_BU_2_df=[]
v_curr_2_df=[]
v_period_2_df = []

for i in range(len(v_un_BU)):
    this_BU = v_un_BU[i]
    this_curr = v_un_curr[i]
    
    for period in list_future_periods:
        v_BU_2_df.append(this_BU)
        v_curr_2_df.append(this_curr)
        v_period_2_df.append(period)

    

In [40]:
list_all_columns = df_billings.columns
list_all_columns

Index(['curr', 'BU', 'period', 'recognized_DC', 'recognized_US', 'service_DC',
       'service_US', 'deferred_B_DC', 'deferred_B_US', 'deferred_1M_DC',
       'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC', 'deferred_2Y_DC',
       'deferred_3Y_DC', 'deferred_1M_US', 'deferred_3M_US', 'deferred_6M_US',
       'deferred_1Y_US', 'deferred_2Y_US', 'deferred_3Y_US', 'Period_Weeks'],
      dtype='object')

In [41]:
list_keepers= []
for i in list_all_columns:
    
    if i[-2:]=='DC':
        print(i)
        list_keepers.append(i)

list_keepers

recognized_DC
service_DC
deferred_B_DC
deferred_1M_DC
deferred_3M_DC
deferred_6M_DC
deferred_1Y_DC
deferred_2Y_DC
deferred_3Y_DC


['recognized_DC',
 'service_DC',
 'deferred_B_DC',
 'deferred_1M_DC',
 'deferred_3M_DC',
 'deferred_6M_DC',
 'deferred_1Y_DC',
 'deferred_2Y_DC',
 'deferred_3Y_DC']

In [42]:
df_fcst = pd.DataFrame({'curr': v_curr_2_df,
                        'BU': v_BU_2_df,
                       'period': v_period_2_df})

In [43]:
for col in list_keepers:
    df_fcst[col]=0


In [44]:
df_fcst.head(5)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC
0,ARS,Creative,2020-04,0,0,0,0,0,0,0,0,0
1,ARS,Creative,2020-05,0,0,0,0,0,0,0,0,0
2,ARS,Creative,2020-06,0,0,0,0,0,0,0,0,0
3,ARS,Creative,2020-07,0,0,0,0,0,0,0,0,0
4,ARS,Creative,2020-08,0,0,0,0,0,0,0,0,0


## Adding the period weeks to the df_fcst

In [47]:
df_cal_2_merge = df_ADBE_cal.copy()
df_cal_2_merge.drop(['Year', 'Quarter', 'Period', 'Qtr_Ticker', 'Qtr_Start', 'Qtr_End', 'Per_Start',
                     'Per_Ticker','Per_End'], axis=1, inplace=True)

In [49]:
df_fcst = df_fcst.merge(df_cal_2_merge, how='left', left_on='period', right_on='period_match')
df_fcst.drop(['period_match'], axis=1, inplace=True)
df_fcst.head(20)

,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,Period_Weeks,period_match
0,ARS,Creative,2020-04,0,0,0,0,0,0,0,0,0,4,2020-04
1,ARS,Creative,2020-05,0,0,0,0,0,0,0,0,0,4,2020-05
2,ARS,Creative,2020-06,0,0,0,0,0,0,0,0,0,5,2020-06
3,ARS,Creative,2020-07,0,0,0,0,0,0,0,0,0,4,2020-07
4,ARS,Creative,2020-08,0,0,0,0,0,0,0,0,0,4,2020-08
5,ARS,Creative,2020-09,0,0,0,0,0,0,0,0,0,5,2020-09
6,ARS,Creative,2020-10,0,0,0,0,0,0,0,0,0,4,2020-10
7,ARS,Creative,2020-11,0,0,0,0,0,0,0,0,0,4,2020-11
8,ARS,Creative,2020-12,0,0,0,0,0,0,0,0,0,5,2020-12
9,ARS,Creative,2021-01,0,0,0,0,0,0,0,0,0,5,2021-01


In [51]:
df_fcst.drop(['period_match'], axis=1, inplace=True)
df_fcst.head(10)


,curr,BU,period,recognized_DC,service_DC,deferred_B_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,Period_Weeks
0,ARS,Creative,2020-04,0,0,0,0,0,0,0,0,0,4
1,ARS,Creative,2020-05,0,0,0,0,0,0,0,0,0,4
2,ARS,Creative,2020-06,0,0,0,0,0,0,0,0,0,5
3,ARS,Creative,2020-07,0,0,0,0,0,0,0,0,0,4
4,ARS,Creative,2020-08,0,0,0,0,0,0,0,0,0,4
5,ARS,Creative,2020-09,0,0,0,0,0,0,0,0,0,5
6,ARS,Creative,2020-10,0,0,0,0,0,0,0,0,0,4
7,ARS,Creative,2020-11,0,0,0,0,0,0,0,0,0,4
8,ARS,Creative,2020-12,0,0,0,0,0,0,0,0,0,5
9,ARS,Creative,2021-01,0,0,0,0,0,0,0,0,0,5


In [ ]:
# starting with the 3 year billings
list_future_periods

In [ ]:
list_bill_periods

In [ ]:
list_bill_periods[-36]

In [ ]:
for i in range(len(v_curr_2_df)):
    this_BU = v_BU_2_df[i]
    this_curr = v_curr_2_df[i]
   
    
    for i in range(len(list_future_periods)):
        this_period = list_future_periods[i]
        this_past_period = list_bill_periods[-36+i]
        print('This period: ', this_period)
        print("This past period: ", this_past_period)
    
    # df_fcst['curr', 'BU', this_peroid] = df_billings['curr', 'BU', this_past_period]
    

In [ ]:
for i in range(len(v_BU_2_df)):
    this_BU = v_BU_2_df[i]
    this_curr = v_curr_2_df[i]
    print(this_BU, this_curr)
    
    df_slice = df_billings[(df_billings['BU']==this_BU) &
                            (df_billings['curr']== this_curr)].copy()
    
    for j in range(len(list_future_periods)):
        this_period = list_future_periods[j]
        
        old_per_3Y = list_bill_periods[-36+j]
        old_per_2Y = list_bill_periods[-24+j]
        old_per_1Y = list_bill_periods[-12+j]
        #old_per_1M = 
        
        # three year
        if old_per_3Y in df_slice['period'].values:
            this_amount = df_slice.loc[df_billings['period']==old_per_3Y, 'deferred_3Y_DC'].copy()
            #print(this_amount)
        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_3Y_DC'] = this_amount.values
        
        # two years
        if old_per_2Y in df_slice['period'].values:
            this_amount = df_slice.loc[df_billings['period']==old_per_2Y, 'deferred_2Y_DC'].copy()
            #print(this_amount)
        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_2Y_DC'] = this_amount.values
        # one year
        if old_per_1Y in df_slice['period'].values:
            this_amount = df_slice.loc[df_billings['period']==old_per_1Y, 'deferred_1Y_DC'].copy()
            #print(this_amount)
        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_1Y_DC'] = this_amount.values
            
        # six months
        if j <6:
            old_per_6M = list_bill_periods[-6+j]
            if old_per_6M in df_slice['period'].values:
                this_amount = df_slice.loc[df_billings['period']==old_per_6M, 'deferred_6M_DC'].copy()
                #print(this_amount)
            
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_6M_DC'] = this_amount.values
            
        else: #case where we are beyond six month
                # reset the index to be based off current df_fcst dataframe 6 months prior
                temp_6M = list_future_periods[j-6]
                this_amount = df_fcst.loc[(df_fcst['BU']==this_BU)&
                                          (df_fcst['curr']==this_curr)&
                                          (df_fcst['period']==temp_6M), 'deferred_6M_DC']
                                          
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_6M_DC'] = this_amount.values
            
        # three months:
        if j <3:
            old_per_3M = list_bill_periods[-3+j]
            if old_per_3M in df_slice['period'].values:
                this_amount = df_slice.loc[df_billings['period']==old_per_3M, 'deferred_3M_DC'].copy()
                #print(this_amount)
            
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_3M_DC'] = this_amount.values
            
        else: #case where we are beyond six month
                # reset the index to be based off current df_fcst dataframe 6 months prior
                temp_3M = list_future_periods[j-3]
                #print(temp_3M, this_period)
                this_amount = df_fcst.loc[(df_fcst['BU']==this_BU)&
                                          (df_fcst['curr']==this_curr)&
                                          (df_fcst['period']==temp_3M), 'deferred_3M_DC']
                                          
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_3M_DC'] = this_amount.values
        
        



In [ ]:
this_amount = df_fcst.loc[(df_fcst['BU']==this_BU)&
                                          (df_fcst['curr']==this_curr)&
                                          (df_fcst['period']==temp_3M), 'deferred_3M_DC']
                               
this_amount.values

In [ ]:
df_slice

In [ ]:
if this_past_period in df_slice['period'].values:
    print('yes')

In [ ]:
df_slice.head(60)

In [ ]:
test_output = df_fcst[(df_fcst['curr']=='USD')&
                     (df_fcst['BU']=='Creative')]
test_output.head(20)

In [ ]:
test_bill = df_billings[(df_billings['curr']=='USD') &
                        (df_billings['BU']=='Creative')]
test_bill.head(63)

## Now working on the monthly model

In [ ]:
df_billings.columns

In [ ]:
for i in range(len(v_BU_2_df)):
    this_BU = v_BU_2_df[i]
    this_curr = v_curr_2_df[i]
    print(this_BU, this_curr)
    
    df_slice = df_billings[(df_billings['BU']==this_BU) &
                            (df_billings['curr']== this_curr)].copy()
    
    # need to determine if we have the entire dataset of monthly billings (no periods missing)
    
    df_slice.sort(['period'],inplace=True)
    X = df_slice['deferred_1M_DC']/df_slice['']
    
    
    
    
    
    for j in range(len(list_future_periods)):
        this_period = list_future_periods[j]
        
        old_per_3Y = list_bill_periods[-36+j]
        old_per_2Y = list_bill_periods[-24+j]
        old_per_1Y = list_bill_periods[-12+j]
        #old_per_1M = 
        
        # three year
        if old_per_3Y in df_slice['period'].values:
            this_amount = df_slice.loc[df_billings['period']==old_per_3Y, 'deferred_3Y_DC'].copy()
            #print(this_amount)
        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_3Y_DC'] = this_amount.values
        
        # two years
        if old_per_2Y in df_slice['period'].values:
            this_amount = df_slice.loc[df_billings['period']==old_per_2Y, 'deferred_2Y_DC'].copy()
            #print(this_amount)
        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_2Y_DC'] = this_amount.values
        # one year
        if old_per_1Y in df_slice['period'].values:
            this_amount = df_slice.loc[df_billings['period']==old_per_1Y, 'deferred_1Y_DC'].copy()
            #print(this_amount)
        
            df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_1Y_DC'] = this_amount.values
            
        # six months
        if j <6:
            old_per_6M = list_bill_periods[-6+j]
            if old_per_6M in df_slice['period'].values:
                this_amount = df_slice.loc[df_billings['period']==old_per_6M, 'deferred_6M_DC'].copy()
                #print(this_amount)
            
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_6M_DC'] = this_amount.values
            
        else: #case where we are beyond six month
                # reset the index to be based off current df_fcst dataframe 6 months prior
                temp_6M = list_future_periods[j-6]
                this_amount = df_fcst.loc[(df_fcst['BU']==this_BU)&
                                          (df_fcst['curr']==this_curr)&
                                          (df_fcst['period']==temp_6M), 'deferred_6M_DC']
                                          
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_6M_DC'] = this_amount.values
            
        # three months:
        if j <3:
            old_per_3M = list_bill_periods[-3+j]
            if old_per_3M in df_slice['period'].values:
                this_amount = df_slice.loc[df_billings['period']==old_per_3M, 'deferred_3M_DC'].copy()
                #print(this_amount)
            
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_3M_DC'] = this_amount.values
            
        else: #case where we are beyond six month
                # reset the index to be based off current df_fcst dataframe 6 months prior
                temp_3M = list_future_periods[j-3]
                #print(temp_3M, this_period)
                this_amount = df_fcst.loc[(df_fcst['BU']==this_BU)&
                                          (df_fcst['curr']==this_curr)&
                                          (df_fcst['period']==temp_3M), 'deferred_3M_DC']
                                          
                df_fcst.loc[(df_fcst['BU']==this_BU)&
                            (df_fcst['curr']==this_curr)&
                            (df_fcst['period']==this_period), 'deferred_3M_DC'] = this_amount.values
        
        

